# This notebook will be mainly used for the capstone project

In [1]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
import numpy as np
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [2]:
!wget -O accident_severity.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-09-29 10:30:46--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘accident_severity.csv’

100%[======================================>] 73,917,638  35.0MB/s   in 2.0s   

2020-09-29 10:30:49 (35.0 MB/s) - ‘accident_severity.csv’ saved [73917638/73917638]



In [3]:
accident = pd.read_csv('accident_severity.csv', low_memory=False)
accident.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [4]:
accident.drop(['SEVERITYCODE.1'], axis='columns', inplace=True)
accident.shape

(194673, 37)

In [5]:
accident.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: object

In [32]:
accident.groupby(['SPEEDING','SEVERITYCODE'])['SEVERITYCODE'].count()

SPEEDING  SEVERITYCODE
Y         1               5802
          2               3531
Name: SEVERITYCODE, dtype: int64

In [6]:
# Categorization
accident['WEATHER'] = pd.Categorical(accident['WEATHER'])
accident['ROADCOND'] = pd.Categorical(accident['ROADCOND'])
accident['LIGHTCOND'] = pd.Categorical(accident['LIGHTCOND'])
accident['LOCATION'] = pd.Categorical(accident['LOCATION'])
accident['SPEEDING'] = pd.Categorical(accident['SPEEDING'])

accident['WEATHER_CODE'] = accident['WEATHER'].cat.codes
accident['ROADCOND_CODE'] = accident['ROADCOND'].cat.codes
accident['LIGHTCOND_CODE'] = accident['LIGHTCOND'].cat.codes
accident['LOCATION_CODE'] = accident['LOCATION'].cat.codes
accident['SPEEDING_CODE'] = accident['SPEEDING'].cat.codes

In [7]:
accident.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEATHER_CODE,ROADCOND_CODE,LIGHTCOND_CODE,LOCATION_CODE,SPEEDING_CODE
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,10,Entering at angle,0,0,N,4,8,5,8793,-1
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,11,From same direction - both going straight - bo...,0,0,N,6,8,2,10707,-1
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,32,One parked--one moving,0,0,N,4,0,5,8049,-1
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,23,From same direction - all others,0,0,N,1,0,5,4647,-1
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,10,Entering at angle,0,0,N,6,8,5,22787,-1


In [8]:
X = np.asarray(accident[['WEATHER_CODE','ROADCOND_CODE','LIGHTCOND_CODE','LOCATION_CODE','SPEEDING_CODE']])
X[0:5]

array([[    4,     8,     5,  8793,    -1],
       [    6,     8,     2, 10707,    -1],
       [    4,     0,     5,  8049,    -1],
       [    1,     0,     5,  4647,    -1],
       [    6,     8,     5, 22787,    -1]], dtype=int16)

In [9]:
y = np.asarray(accident[['SEVERITYCODE']])
y[0:5]

array([[2],
       [1],
       [1],
       [1],
       [2]])

In [10]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 0.35364615,  1.50545441,  0.3912104 , -0.45743913, -0.22440165],
       [ 1.04520829,  1.50545441, -1.18714134, -0.17720325, -0.22440165],
       [ 0.35364615, -0.68713674,  0.3912104 , -0.56637095, -0.22440165],
       [-0.68369706, -0.68713674,  0.3912104 , -1.06447047, -0.22440165],
       [ 1.04520829,  1.50545441,  0.3912104 ,  1.59147464, -0.22440165]])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (136271, 5) (136271, 1)
Test set: (58402, 5) (58402, 1)


# Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [13]:
yhat_lr = LR.predict(X_test)
yhat_lr[0:5]

array([1, 1, 1, 1, 1])

In [14]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

array([[0.73023109, 0.26976891],
       [0.68054332, 0.31945668],
       [0.65708122, 0.34291878],
       ...,
       [0.58469252, 0.41530748],
       [0.8187009 , 0.1812991 ],
       [0.68958664, 0.31041336]])

In [15]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [16]:
print("Jaccard: ",jaccard_similarity_score(y_test, yhat_lr))
print("F1 Score: ",f1_score(y_test,yhat_lr,average='weighted'))
print("LogLoss :",log_loss(y_test, yhat_prob))

Jaccard:  0.7030581144481354
F1 Score:  0.5812429210363583
LogLoss : 0.6003036801500582


# Decision Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion="entropy", max_depth=4)
dt.fit(X_train, y_train)
dt

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [18]:
yhat_dt = dt.predict(X_test)
yhat_dt[0:5]

array([1, 1, 1, 1, 1])

In [19]:
print("F1 Score: ",f1_score(y_test,yhat_dt,average='weighted'))
print("Jaccard: ",jaccard_similarity_score(y_test, yhat_dt))

F1 Score:  0.5809904188654375
Jaccard:  0.7034519365775145


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
